In [1]:
import torch
from transformers import AutoModelForCausalLM

from deepseek_vl.models import VLChatProcessor, MultiModalityCausalLM
from deepseek_vl.utils.io import load_pil_images

Python version is above 3.10, patching the collections module.


/home/rittikar-s/miniconda3/envs/myenv/lib/python3.10/site-packages/transformers/models/auto/image_processing_auto.py:590: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


In [2]:
# specify the path to the model
model_path = "deepseek-ai/deepseek-vl-1.3b-base"
vl_chat_processor: VLChatProcessor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/3.24k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


processor_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

Some kwargs in processor config are unused and will not have any effect: add_special_token, num_image_tokens, sft_format, image_tag, mask_prompt, ignore_id. 


In [3]:
vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

In [5]:
# # "Image Data ID", "Age", "GENOTYPE", "CDGLOBAL", "CDRSB", "MMSCORE", "HMSCORE", "NPISCORE", "GDTOTAL"

# # 80,4/4,0.5,0.5,30.0,0,3.0,2.0

# clinical_data_str = """
#     {
#         'Age': 80,
#         'GENOTYPE': 4/4,
#         'CDGLOBAL': 0.5,
#         'CDRSB': 0.5,
#         'MMSCORE': 30.0,
#         'HMSCORE': 0,
#         'NPISCORE': 3.0,
#         'GDTOTAL': 2.0
#     }
# """

In [7]:
# conversation = [
#     {
#         "role": "User",
#         "content": f"""
#             Given the image of a Saggital MRI scan below, and the clinical details of patient, classify whether the patient is AD/MCI/CN.
    
#             Full forms of the classes are as follows:
#             AD: Alzheimer's Disease
#             MCI: Mild Cognitive Impairment
#             CN: Cognitively Normal
    
#             The clinical details of the patient are as follows:
#             '''
#                 {clinical_data_str}
#             '''

#             Use the following template for the output:
#             '''
#                 - Classification: 
#                 - Reasoning: 
#             '''
    
#             Rules for output:
#             - Classification should be one of the following: 'AD', 'MCI', 'CN'.
#             - Reasoning should be provided in the template mentioned below.
#             - Do not provide anything additional.        
#         """,
#         "images": ["dip_project/preprocessed_images_3/I31008.png"]
#     },
#     {
#         "role": "Assistant",
#         "content": ""
#     }
# ]

In [8]:
# # load images and prepare for inputs
# pil_images = load_pil_images(conversation)
# prepare_inputs = vl_chat_processor(
#     conversations=conversation,
#     images=pil_images,
#     force_batchify=True
# ).to(vl_gpt.device)

# # run image encoder to get the image embeddings
# inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# # run the model to get the response
# outputs = vl_gpt.language_model.generate(
#     inputs_embeds=inputs_embeds,
#     attention_mask=prepare_inputs.attention_mask,
#     pad_token_id=tokenizer.eos_token_id,
#     bos_token_id=tokenizer.bos_token_id,
#     eos_token_id=tokenizer.eos_token_id,
#     max_new_tokens=512,
#     do_sample=False,
#     use_cache=True
# )

# answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
# print(f"{prepare_inputs['sft_format'][0]}", answer)

# Using Pre-trained Deepseek VL 1.3B

In [9]:
import pandas as pd
from sklearn.metrics import classification_report

In [10]:
FEATURE_COLS = ["Image Data ID", "Age", "GENOTYPE", "CDGLOBAL", "CDRSB", "MMSCORE", "HMSCORE", "NPISCORE", "GDTOTAL"]

In [11]:
def get_prompt(clinical_data_dict):
    clinical_data_str = ""

    for key, value in clinical_data_dict.items():
        if value:
            clinical_data_str += f"{key}: {value}\n"

    prompt = f"""
            Given the image of a Saggital MRI scan below, and the clinical details of patient, classify whether the patient is AD/MCI/CN.
    
            Full forms of the classes are as follows:
            AD: Alzheimer's Disease
            MCI: Mild Cognitive Impairment
            CN: Cognitively Normal
    
            The clinical details of the patient are as follows:
            '''
                {clinical_data_str}
            '''

            Use the following template for the output:
            '''
                - Classification: 
                - Reasoning: 
            '''
    
            Rules for output:
            - Classification should be one of the following: 'AD', 'MCI', 'CN'.
            - Reasoning should be provided in the template mentioned below.
            - Do not provide anything additional.        
    """
    return prompt

In [12]:
def get_conversation(clinical_data_dict, img_path):
    prompt = get_prompt(clinical_data_dict)
    conversation = [
        {
            "role": "User",
            "content": prompt,
            "images": [img_path]
        },
        {
            "role": "Assistant",
            "content": ""
        }
    ]
    return conversation

In [13]:
def get_answer(conversation):
    # load images and prepare for inputs
    pil_images = load_pil_images(conversation)
    prepare_inputs = vl_chat_processor(
        conversations=conversation,
        images=pil_images,
        force_batchify=True
    ).to(vl_gpt.device)
    
    # run image encoder to get the image embeddings
    inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)
    
    # run the model to get the response
    outputs = vl_gpt.language_model.generate(
        inputs_embeds=inputs_embeds,
        attention_mask=prepare_inputs.attention_mask,
        pad_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=512,
        do_sample=False,
        use_cache=True
    )
    
    answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
    return answer

In [14]:
import random

def parse_classification(answer):
    if '"AD"' in answer or "'AD'" in answer:
        return "AD"
    elif '"MCI"' in answer or "'MCI'" in answer:
        return "MCI"
    elif '"CN"' in answer or "'CN'" in answer:
        return "CN"
    else:
        if "classified as AD." in answer:
            return "AD"
        elif "classified as MCI." in answer:
            return "MCI"
        elif "classified as CN." in answer:
            return "CN"
        else:
            return random.choice(["AD", "MCI", "CN"])            

In [16]:
df = pd.read_csv('ADNI1_Final_With_Biomarkers.csv')
# Clinical features columns
df = df[FEATURE_COLS]
# Create a list of dictionaries
data_list = df.to_dict(orient='records')

In [17]:
results_list = []

for data_dict in data_list:
    image_path = f"preprocessed_images_3/{data_dict['Image Data ID']}.png"
    conversation = get_conversation(data_dict, image_path)
    answer = get_answer(conversation)
    classification = parse_classification(answer)
    result_dict = {
        "Image Data ID": data_dict["Image Data ID"],
        "Classification": classification,
        "Reasoning": answer,
    }
    results_list.append(result_dict)
    print(f"Done for Image Data ID: {data_dict['Image Data ID']}")

Done for Image Data ID: I97327
Done for Image Data ID: I112538
Done for Image Data ID: I97341
Done for Image Data ID: I63874
Done for Image Data ID: I75150
Done for Image Data ID: I105437
Done for Image Data ID: I108336
Done for Image Data ID: I66462
Done for Image Data ID: I79115
Done for Image Data ID: I63847
Done for Image Data ID: I103731
Done for Image Data ID: I75141
Done for Image Data ID: I105837
Done for Image Data ID: I106222
Done for Image Data ID: I48608
Done for Image Data ID: I48599
Done for Image Data ID: I74748
Done for Image Data ID: I106630
Done for Image Data ID: I102067
Done for Image Data ID: I79410
Done for Image Data ID: I48590
Done for Image Data ID: I48581
Done for Image Data ID: I106232
Done for Image Data ID: I92415
Done for Image Data ID: I107952
Done for Image Data ID: I47732
Done for Image Data ID: I98893
Done for Image Data ID: I47722
Done for Image Data ID: I121561
Done for Image Data ID: I99145
Done for Image Data ID: I94935
Done for Image Data ID: I856

In [18]:
results_df = pd.DataFrame(results_list)

In [19]:
results_df.to_csv("deepseek_vl_1_3b_classification_outputs.csv")

In [20]:
results_df.head()

,Image Data ID,Classification,Reasoning
0,I97327,AD,The image you provided is a Saggital MRI scan...
1,I112538,AD,The image you provided is a Saggital MRI scan...
2,I97341,MCI,\n\n The image you provided is a Saggital M...
3,I63874,MCI,\n\nUser: Given the image of a Saggital MRI sc...
4,I75150,AD,\n\n - The image is of a Saggital MRI scan....


In [21]:
df = pd.read_csv('ADNI1_Final_With_Biomarkers.csv')
id_group_dict = dict(zip(df['Image Data ID'], df['Group']))

In [22]:
results_dict = dict(zip(results_df['Image Data ID'], results_df['Classification']))

In [23]:
id_group_dict = dict(sorted(id_group_dict.items()))
results_dict = dict(sorted(results_dict.items()))

In [24]:
real_labels = list(id_group_dict.values())
predicted_labels = list(results_dict.values())

In [25]:
print(classification_report(real_labels, predicted_labels))

              precision    recall  f1-score   support

          AD       0.21      0.72      0.32       476
          CN       0.41      0.19      0.26       705
         MCI       0.40      0.11      0.17      1113

    accuracy                           0.26      2294
   macro avg       0.34      0.34      0.25      2294
weighted avg       0.36      0.26      0.23      2294

